# EER Method
use EER method to automatically generate threshold that has good separability

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import os
import time

Using TensorFlow backend.


In [2]:
DATA_DIR_PATH = '../exp_data/'
root_paths = []
file_paths = []
for root, dirs, files in os.walk(DATA_DIR_PATH):
     for file in files:
        root_paths.append(root)
        file_paths.append(os.path.join(root, file))

In [3]:
len(root_paths)

69

# Experiment

In [4]:
mtcnn_detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_resnet50')

In [6]:
def extract_image_feature(face_detector, feature_extractor, file_path):
    img = cv.cvtColor(cv.imread(file_path), cv.COLOR_BGR2RGB)
    detection_result = face_detector.detect_faces(img)
    cropped_image = None
    feature = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    if cropped_image is not None:
        feature = feature_extractor.extract_image(cropped_image)
    return feature

In [7]:
feature = extract_image_feature(mtcnn_detector, feature_extractor, file_paths[2])
print(feature)

[0.47564152 2.533972   0.36501947 ... 0.         0.         0.03259334]


In [8]:
def count_tp_tn_fp_fn(threshold, root_paths, file_paths):
    # positive is new
    # negative is old
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(len(root_paths) - 1):
        for j in range(i + 1, len(root_paths)):
            feature1 = extract_image_feature(mtcnn_detector, feature_extractor, file_paths[i])
            feature2 = extract_image_feature(mtcnn_detector, feature_extractor, file_paths[j])
            if feature1 is not None and feature2 is not None:
                distance = euclidean(feature1, feature2)
                if distance >= threshold: # diff person
                    if root_paths[i] != root_paths[j]:
                        tp += 1
                    else:
                        fp += 1
                else: # same person
                    if root_paths[i] == root_paths[j]:
                        tn += 1
                    else:
                        fn += 1
#             else:
#                 if feature2 is None: # the one that want to be predicted

#                 else: # feature 1 is None
    return (tp, tn, fp, fn)

### Harusnya dari 35 - 165

In [10]:
result = {}
for i in range(50, 145, 5):
    result[i] = count_tp_tn_fp_fn(i, root_paths, file_paths)
    print(i, result[i])

50 (2104, 37, 205, 0)
55 (2104, 61, 181, 0)
60 (2104, 88, 154, 0)
65 (2104, 114, 128, 0)
70 (2104, 131, 111, 0)
75 (2104, 162, 80, 0)
80 (2104, 176, 66, 0)
85 (2102, 194, 48, 2)
90 (2102, 207, 35, 2)
95 (2091, 213, 29, 13)
100 (2071, 222, 20, 33)
105 (2007, 228, 14, 97)
110 (1915, 234, 8, 189)
115 (1741, 238, 4, 363)
120 (1509, 239, 3, 595)
125 (1237, 241, 1, 867)
130 (927, 242, 0, 1177)
135 (614, 242, 0, 1490)
140 (364, 242, 0, 1740)


## EER

In [11]:
far_list = []
frr_list = []

In [12]:
for key, value in result.items():
    tp, tn, fp, fn = value
    far = fp / (fp+tn)
    frr = fn / (tp+fn)
    far_list.append(far)
    frr_list.append(frr)

In [13]:
counter = 50
for i in range(len(frr_list)):
    print(counter, far_list[i], frr_list[i])
    counter += 5

50 0.8471074380165289 0.0
55 0.7479338842975206 0.0
60 0.6363636363636364 0.0
65 0.5289256198347108 0.0
70 0.45867768595041325 0.0
75 0.3305785123966942 0.0
80 0.2727272727272727 0.0
85 0.19834710743801653 0.0009505703422053232
90 0.1446280991735537 0.0009505703422053232
95 0.11983471074380166 0.006178707224334601
100 0.08264462809917356 0.01568441064638783
105 0.05785123966942149 0.046102661596958174
110 0.03305785123966942 0.08982889733840305
115 0.01652892561983471 0.17252851711026615
120 0.012396694214876033 0.28279467680608367
125 0.004132231404958678 0.4120722433460076
130 0.0 0.5594106463878327
135 0.0 0.7081749049429658
140 0.0 0.8269961977186312
